# OpenAI Function Calling In LangChain

In [1]:
from aisModels_lc import GroqChatLLM
llm = GroqChatLLM()

Hello. How can I assist you today?
Hello! How are you today? Is there something I can help you with or would you like to chat?


In [2]:
%%script true 
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
import os
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [4]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [5]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [6]:
foo.name

'Joe'

In [7]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [8]:
foo.age

'bar'

In [9]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [10]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [11]:
foo_p.name

'Jane'

**Note**: The next cell is expected to fail.

In [12]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing

In [13]:
class Class(BaseModel):
    students: List[pUser]

In [14]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [15]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [16]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [17]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [18]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/tmp/ipykernel_8049/1313296718.py:1: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [19]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [20]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

**Note**: The next cell is expected to generate an error.

In [21]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

No args description Field validation

In [22]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [23]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [24]:
#from langchain.chat_models import ChatOpenAI

In [25]:
model = llm #ChatOpenAI()

In [26]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code": "KSFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 245, 'total_tokens': 262, 'completion_time': 0.06830646, 'prompt_time': 0.066405052, 'queue_time': 0.005737490999999997, 'total_time': 0.134711512}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'function_call', 'logprobs': None}, id='run-f8d8ccfa-f5f7-45ca-b50a-d87563aea3fa-0', usage_metadata={'input_tokens': 245, 'output_tokens': 17, 'total_tokens': 262})

In [27]:
model_with_function = model.bind(functions=[weather_function])

In [28]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code": "SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 244, 'total_tokens': 261, 'completion_time': 0.068, 'prompt_time': 0.060424219, 'queue_time': 0.005175300999999993, 'total_time': 0.128424219}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-fc76f658-39e5-4d74-b38e-a5c576fa8434-0', usage_metadata={'input_tokens': 244, 'output_tokens': 17, 'total_tokens': 261})

## Forcing it to use a function

We can force the model to use a function

In [29]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [30]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code": "SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 263, 'total_tokens': 275, 'completion_time': 0.048019499, 'prompt_time': 0.065132195, 'queue_time': 0.005471838999999992, 'total_time': 0.113151694}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-8272520e-26e0-4fdb-a87d-9c2bbb3d3c5c-0', usage_metadata={'input_tokens': 263, 'output_tokens': 12, 'total_tokens': 275})

In [31]:
model_with_forced_function.invoke("hi!").content

''

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [32]:
from langchain.prompts import ChatPromptTemplate

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [34]:
chain = prompt | model_with_function

In [35]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code": "SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 249, 'total_tokens': 266, 'completion_time': 0.068, 'prompt_time': 0.06592434, 'queue_time': 0.109937764, 'total_time': 0.13392434}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-05026044-358b-417b-8a7d-4a1b214e17fb-0', usage_metadata={'input_tokens': 249, 'output_tokens': 17, 'total_tokens': 266})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [36]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [37]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [38]:
model_with_functions = model.bind(functions=functions)

In [39]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code": "SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 332, 'total_tokens': 349, 'completion_time': 0.068, 'prompt_time': 0.082894903, 'queue_time': 0.004246128999999987, 'total_time': 0.150894903}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'function_call', 'logprobs': None}, id='run-b1bddf13-5faa-4139-b206-1d8ef110b6cc-0', usage_metadata={'input_tokens': 332, 'output_tokens': 17, 'total_tokens': 349})

In [40]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name": "Taylor Swift", "n": 3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 334, 'total_tokens': 356, 'completion_time': 0.088628517, 'prompt_time': 0.095371248, 'queue_time': 0.012110202, 'total_time': 0.183999765}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'function_call', 'logprobs': None}, id='run-20048bba-7f3d-48e2-a125-5b1f7dea2017-0', usage_metadata={'input_tokens': 334, 'output_tokens': 22, 'total_tokens': 356})

In [41]:
model_with_functions.invoke("hi!")

AIMessage(content="I'm happy to chat with you. What can I help you with today?", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 327, 'total_tokens': 344, 'completion_time': 0.068, 'prompt_time': 0.081398035, 'queue_time': 0.0053780570000000055, 'total_time': 0.149398035}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None}, id='run-a145186d-02e6-459f-aed5-8351d5dd1720-0', usage_metadata={'input_tokens': 327, 'output_tokens': 17, 'total_tokens': 344})